In [ ]:
"""

Compare the melt resulting from old and new tuning for last 10 years of
OPM026,OPM031,ctrl94,isf94

"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
    
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
mod = 'ACCESS-CM2'
scenario = 'historical'

to2300 = False

if scenario == 'historical':
    yystart = 1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

READ IN OCEAN

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNxsmall96',
              'NNsmall','NNlarge']

In [ ]:
## Melt outputpath
outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

melt1D_list = []

for mparam in param_classic_list:
    
    melt1D_scenario_list = []
    for scenario in ['historical','ssp585']:
        melt_1D = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')
        melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
    melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
    melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
melt1D_classic = xr.concat(melt1D_list, dim='param')       
Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
box1_classic = melt1D_classic['melt_1D_mean_myr_box1']

In [ ]:
melt1D_list = []
for mparam in param_NN_list:
    
    melt1D_scenario_list = []
    for scenario in ['historical','ssp585']:
        melt_1D = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'.nc')
        melt1D_scenario_list.append(melt_1D.assign_coords({'scenario': scenario}))
    melt1D_scenario = xr.concat(melt1D_scenario_list, dim='scenario')
    melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
melt1D_NN = xr.concat(melt1D_list, dim='param')   
Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')

Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
box1_all = xr.concat([box1_classic, box1_NN], dim='param')

#melt_new_NN_list = []
#melt_old_NN_list = []
#for nn_type in ['xsmall96','small','large']:
#    melt_newtuning_nntype = xr.open_mfdataset(outputpath_melt+'evalmetrics_1D_'+nn_type+'_*_newtuning.nc')
#    melt_newtuning_nntype = melt_newtuning_nntype.sel(time=melt_newtuning_nntype.time[-10::]).load()
#    melt_oldtuning_nntype = xr.open_mfdataset(outputpath_melt+'evalmetrics_1D_'+nn_type+'_*_oldtuning.nc').sel(time=melt_newtuning_nntype.time).load()
#    melt_new_NN_list.append(melt_newtuning_nntype.assign_coords({'param': nn_type}))
#    melt_old_NN_list.append(melt_oldtuning_nntype.assign_coords({'param': nn_type}))
#melt_new_NN = xr.concat(melt_new_NN_list, dim='param')
#ds_melt_param_Gt_new.append(melt_new_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))
#ds_melt_param_box1_new.append(melt_new_NN['predicted_melt'].sel(metrics='Gt').squeeze().drop('metrics'))

READ IN ATMOSPHERE

In [ ]:
inputpath_atmo = '/bettik/kittelc/clara/'

melt_atmo_list = []
for scenario in ['historical','ssp585']:
    melt_atmo_scenario = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'.nc')
    melt_atmo_list.append(melt_atmo_scenario.assign_coords({'scenario': scenario}))
melt_atmo = xr.concat(melt_atmo_list, dim='scenario').sel(Nisf=rignot_isf)
melt_atmo['time'] = melt_atmo['time'].dt.year

READ IN GL FLUX

In [ ]:
inputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'

GL_fluxes = xr.open_dataset(inputpath_GL + 'all_GL_fluxes.nc')

READ IN CALVING FLUX

In [ ]:
inputpath_caving = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
calving_flux = xr.open_dataset(inputpath_caving + 'calving_flux_rignot13.nc')

PLOT OCEAN ONLY

In [ ]:
Gt_all.param

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen','royalblue']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    for n,mparam in enumerate(Gt_all.param):
        
        ax[i].plot(Gt_all.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='historical'), color=colors[n],alpha=0.5,
                                                                  label=mparam.values)
        ax[i].plot(Gt_all.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='ssp585'),color=colors[n], alpha=0.5,
                                                                  label=mparam.values)



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#plt.savefig(plot_path+'melt_compare_params_'+mod+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
        
    ax[i].plot(Gt_all.time, Gt_all.sel(Nisf=kisf,scenario='historical').mean('param'), color='blue',
                                                              label=mparam.values)
    ax[i].plot(Gt_all.time, Gt_all.sel(Nisf=kisf,scenario='ssp585').mean('param'),color='blue',
                                                              label=mparam.values)
    
    for n,mparam in enumerate(Gt_all.param):
        
        ax[i].plot(Gt_all.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='historical'), color='blue',alpha=0.15,
                                                                  label=mparam.values)
        ax[i].plot(Gt_all.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='ssp585'),color='blue',alpha=0.15,
                                                                  label=mparam.values)




    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#plt.savefig(plot_path+'melt_compare_oceanparams_'+mod+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
        
    ax[i].plot(melt_atmo.time, melt_atmo['SMB'].sel(Nisf=kisf,scenario='historical'), color='green', alpha=0.5)
    ax[i].plot(melt_atmo.time, melt_atmo['SMB'].sel(Nisf=kisf,scenario='ssp585'),color='green', alpha=0.5)
    ax[i].plot(melt_atmo.time, melt_atmo['SMBorg'].sel(Nisf=kisf,scenario='historical'), color='limegreen', alpha=0.5)
    ax[i].plot(melt_atmo.time, melt_atmo['SMBorg'].sel(Nisf=kisf,scenario='ssp585'),color='limegreen', alpha=0.5)
    



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_atmo_only_'+mod+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
        
    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='orange')
    ax[i].axhline(y=GL_fluxes['flux_Gt_ref'].sel(Nisf=kisf), color='orange',linestyle='--')

    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP_2300'].sel(Nisf=kisf), color='red')
    ax[i].axhline(y=GL_fluxes['flux_Gt_ref_2300'].sel(Nisf=kisf), color='red',linestyle='--')
    



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'GL_flux_only.pdf')

In [ ]:
mass_balance = Gt_all - melt_atmo['SMB']

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].plot(melt_atmo.time, Gt_all.sel(Nisf=kisf,scenario='historical').mean('param').sel(time=melt_atmo.time), color='blue',
                                                              label=mparam.values)
    ax[i].plot(melt_atmo.time, Gt_all.sel(Nisf=kisf,scenario='ssp585').mean('param').sel(time=melt_atmo.time),color='blue',
                                                              label=mparam.values)
    
    for n,mparam in enumerate(Gt_all.param):
        
        ax[i].plot(melt_atmo.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='historical').sel(time=melt_atmo.time), color='blue',alpha=0.15,
                                                                  label=mparam.values)
        ax[i].plot(melt_atmo.time, Gt_all.sel(param=mparam, 
                                            Nisf=kisf,scenario='ssp585').sel(time=melt_atmo.time),color='blue',alpha=0.15,
                                                                  label=mparam.values)
    
    ax[i].plot(melt_atmo.time, -1*melt_atmo['SMB'].sel(Nisf=kisf,scenario='historical'), color='green')
    ax[i].plot(melt_atmo.time, -1*melt_atmo['SMB'].sel(Nisf=kisf,scenario='ssp585'),color='green')
        
    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='red')
    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP_2300'].sel(Nisf=kisf), color='red',linestyle='--')




    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'allfluxes_together_'+mod+'.pdf')

In [ ]:
calving_flux

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*2, 8.25*2)

ax={}

leg_hdl = []

i = 0

mass_balance = Gt_all - melt_atmo['SMB'] - calving_flux['calving_flux_rignot']

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    ax[i].plot(melt_atmo.time, mass_balance.sel(Nisf=kisf,scenario='historical').mean('param').sel(time=melt_atmo['SMB'].time), color='k')
    ax[i].plot(melt_atmo.time, mass_balance.sel(Nisf=kisf,scenario='ssp585').mean('param'),color='k')
    
    for n,mparam in enumerate(Gt_all.param):
        
        ax[i].plot(melt_atmo.time, mass_balance.sel(param=mparam, 
                                            Nisf=kisf,scenario='historical').sel(time=melt_atmo['SMB'].time), color='k',alpha=0.15,
                                                                  label=mparam.values)
        ax[i].plot(melt_atmo.time, mass_balance.sel(param=mparam, 
                                            Nisf=kisf,scenario='ssp585'),color='k',alpha=0.15,
                                                                  label=mparam.values)
    
        
    ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP'].sel(Nisf=kisf), color='red')
    #ax[i].axhline(y=GL_fluxes['flux_Gt_ABUMIP_2300'].sel(Nisf=kisf), color='red',linestyle='--')
    
    ax[i].axvline(x=2014, color='orange',linestyle='--')



    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'massbalance_together_'+mod+'.pdf')

In [ ]:
melt_atmo.time

#############

In [ ]:
simple_param_list = ['linear_local','quadratic_local','quadratic_local_locslope']
complex_param_list = ['lazero19','boxes_4_pismyes_picopno']
nn_param_list = ['xsmall96','small','large']
all_param_list = simple_param_list + complex_param_list + nn_param_list

In [ ]:
GL_fluxes

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0


for kisf in tqdm(sorted_isf_ri):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=simple_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_simpleparams_'+nemo_run+'.pdf')

In [ ]:
ds_melt_param_Gt_new_nrun.param

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['deepskyblue','royalblue']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=complex_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_complexparams_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['limegreen','olive','yellowgreen']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=nn_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],linestyle='--',
                                                               alpha=0.7)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_old_new_tuning_nnparams_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta','deepskyblue','royalblue','limegreen','olive','green']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=all_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_new_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values)


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_onlynew_tuning_'+nemo_run+'.pdf')

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['maroon','orange','magenta','deepskyblue','royalblue','limegreen','olive','green']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

for kisf in tqdm(sorted_isf_all):
    
    ax[i] = f.add_subplot(6,6,i+1)
    ax[i].plot(ds_melt_param_Gt_new_nrun.time, ds_melt_param_Gt_old_nrun.sel(param='reference', Nisf=kisf), color='k')

    
    for n,mparam in enumerate(ds_melt_param_Gt_new_nrun.param.sel(param=all_param_list)):
        
        ax[i].plot(ds_melt_param_Gt_old_nrun.time, ds_melt_param_Gt_old_nrun.sel(param=mparam, 
                                                               Nisf=kisf), color=colors[n],alpha=0.7,
                                                                  label=mparam.values,linestyle='--')


    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
plt.savefig(plot_path+'melt_compare_onlyold_tuning_'+nemo_run+'.pdf')